### 5. Setup

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_operation_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_operation_credential


### 6. Go to hospital notebook to issue credential

### 7. Get the Record

In [4]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'auto_issue': False, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:MedicalRecords:0.0.3', 'role': 'holder', 'updated_at': '2023-01-06 05:11:29.037630Z', 'credential_definition_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:85:default', 'credential_offer': {'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:MedicalRecords:0.0.3', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:85:default', 'key_correctness_proof': {'c': '37658955705197760903476251001339094963830734290645554208638524699278482443719', 'xz_cap': '774744531200510732346208170858994365249173003084413487926366039984863735339313978709005442017098541224475163238467721208628082434745242331564746068077409662208045592137834440091835926896967721979017142249656547555178922202309246391222145240532632673149807292903396957204793000186803333666328729427736804039284337554664283803516139259405421781268346369169520642389481424753840222282806282115886137343554398468844886561201392890444545697257738932430552472299490590749951724369032621169412668837808535855157692

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange acccbefe-2627-4925-b020-273fc9482d07, role: holder, state: offer_received
Being offered: [{'name': 'fullname', 'value': 'tim'}, {'name': 'age', 'value': '21'}, {'name': 'date', 'value': '01/01/23'}, {'name': 'doctorname', 'value': 'amy'}, {'name': 'complaint', 'value': 'no'}, {'name': 'examination results', 'value': 'no'}, {'name': 'final treatment', 'value': 'no'}, {'name': 'medicine', 'value': 'no'}, {'name': 'nextappointment', 'value': 'no'}]


### 8. Send request for record

In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange acccbefe-2627-4925-b020-273fc9482d07, role: holder, state: request_sent


### 9. Store Credential

In [9]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My Sensitive data")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

ClientResponseError: 404, message='credential_exchange_v10 record not found: acccbefe-2627-4925-b020-273fc9482d07. WalletItemNotFound.', url=URL('http://the_wallet-agent:7981/issue-credential/records/acccbefe-2627-4925-b020-273fc9482d07/store')

Error during POST /issue-credential/records/acccbefe-2627-4925-b020-273fc9482d07/store: 404, message='credential_exchange_v10 record not found: acccbefe-2627-4925-b020-273fc9482d07. WalletItemNotFound.', url=URL('http://the_wallet-agent:7981/issue-credential/records/acccbefe-2627-4925-b020-273fc9482d07/store')


In [10]:
emergeny_mode = True
if emergeny_mode:
    credential = await agent_controller.credentials.get_by_id("My Sensitive data")
    print(credential)

{'referent': 'My Sensitive data', 'attrs': {'doctorname': 'amy', 'age': '21', 'date': '01/01/23', 'nextappointment': 'no', 'examination results': 'no', 'complaint': 'no', 'medicine': 'no', 'final treatment': 'no', 'fullname': 'tim'}, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:MedicalRecords:0.0.3', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:85:default', 'rev_reg_id': None, 'cred_rev_id': None}


### **Terminate**

In [11]:
response = await agent_controller.terminate()
print(response)

None


In [ ]:
## TODO emergency mode